In [1]:
from huggingface_hub import login
from google.colab import userdata

In [2]:
login(token=userdata.get('niru_hf_read'))

In [3]:
from huggingface_hub import snapshot_download

model_name = "nirusanan/Mistral_7B_DPO_Finetune_Pandas_Code_Gen_Final"

snapshot_download(repo_id=model_name, local_dir="mistral-hf",
                  local_dir_use_symlinks=False, revision="main")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/9.08G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

'/content/mistral-hf'

In [4]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 42517, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 42517 (delta 4), reused 3 (delta 2), pack-reused 42506 (from 2)
Receiving objects: 100% (42517/42517), 77.00 MiB | 8.17 MiB/s, done.
Resolving deltas: 100% (31065/31065), done.


In [ ]:
!pip install -r llama.cpp/requirements.txt

In [ ]:
!python /content/llama.cpp/convert_hf_to_gguf.py mistral-hf --outfile mistral-hf.gguf --outtype q8_0

In [11]:
!pip install -q llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00


In [12]:
from llama_cpp import Llama

In [13]:
llm = Llama(model_path="/content/mistral-hf.gguf", chat_format="llama-2",n_ctx=850)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from /content/mistral-hf.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Mistral_7B_SFT_Finetune_Pandas_Code_G...
llama_model_loader: - kv   3:                       general.organization str              = Nirusanan
llama_model_loader: - kv   4:                         general.size_label str              = 7.2B
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                       llama.context_length u32              = 32768
llama_model_loader: - kv   7:                  

In [14]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""

output = llm(
  prompt, # Prompt
  max_tokens=850,  # Generate up to 512 tokens
  stop=["### End"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

print(output['choices'][0]['text'])

llama_perf_context_print:        load time =  183087.19 ms
llama_perf_context_print: prompt eval time =  183084.03 ms /   471 tokens (  388.71 ms per token,     2.57 tokens per second)
llama_perf_context_print:        eval time =  129680.29 ms /   109 runs   ( 1189.73 ms per token,     0.84 tokens per second)
llama_perf_context_print:       total time =  312844.31 ms /   580 tokens



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(df):
    #count uniqu

In [ ]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
customer C007 how much total money spent on sales

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""

output = llm(
  prompt, # Prompt
  max_tokens=850,  # Generate up to 512 tokens
  stop=["### End"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =   78016.69 ms
llama_print_timings:      sample time =     164.23 ms /   259 runs   (    0.63 ms per token,  1577.06 tokens per second)
llama_print_timings: prompt eval time =   71325.63 ms /   443 tokens (  161.01 ms per token,     6.21 tokens per second)
llama_print_timings:        eval time =   73771.64 ms /   258 runs   (  285.94 ms per token,     3.50 tokens per second)
llama_print_timings:       total time =  146274.60 ms /   701 tokens



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
customer C007 how much total money spent on sales

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(df)

In [ ]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
which item has high unit price

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""

output = llm(
  prompt, # Prompt
  max_tokens=850,  # Generate up to 512 tokens
  stop=["### End"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =   78016.69 ms
llama_print_timings:      sample time =      61.65 ms /    97 runs   (    0.64 ms per token,  1573.35 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   27462.83 ms /    97 runs   (  283.12 ms per token,     3.53 tokens per second)
llama_print_timings:       total time =   27884.08 ms /    98 tokens



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
which item has high unit price

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(df):
    # Find the it

In [ ]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
Top 5 customers in term of sales amount

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""

output = llm(
  prompt, # Prompt
  max_tokens=850,  # Generate up to 512 tokens
  stop=["### End"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =   78016.69 ms
llama_print_timings:      sample time =     159.83 ms /   249 runs   (    0.64 ms per token,  1557.95 tokens per second)
llama_print_timings: prompt eval time =   72851.05 ms /   440 tokens (  165.57 ms per token,     6.04 tokens per second)
llama_print_timings:        eval time =   70789.60 ms /   248 runs   (  285.44 ms per token,     3.50 tokens per second)
llama_print_timings:       total time =  144786.77 ms /   688 tokens



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
Top 5 customers in term of sales amount

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(df):
    # Gr

In [15]:
from google.colab import userdata
from huggingface_hub import login, HfApi, create_repo

In [16]:
login(token=userdata.get('niru_hf_write'))

In [ ]:
api = HfApi()

model_id = "nirusanan/llama-gguf"

api.upload_file(
    path_or_fileobj="/content/mistral-hf.gguf",
    path_in_repo="Mistral_7B_DPO_Pandas-q8_0.gguf",
    repo_id=model_id,
)